### Setup Train Raster

In [ ]:
train_ref = '/mnt/disk1/jasp/a2autocls2023/urbanatlas/funchal/ua18_gen.shp'

class_col = 'code'

class_proportion = {
    1 : 21.23,
    2 : 38.68,
    3 : 36.28,
    4 : 1.1,
    5 : 0.01,
    6 : 2.71
}

ref_raster = '/mnt/disk1/jasp/a2autocls2023/imgmad/step3_2018/b02_20180812.tif'

clouds = '/mnt/disk1/jasp/a2autocls2023/imgmad/clouds_2018.tif'

sample_dim = 500000
cls_sample_min = 400

out_rst = '/mnt/disk1/jasp/a2autocls2023/imgmad/train/ua18_500k.tif'

In [ ]:
import os
from glass.pys.oss import fprop
from glass.pys.tm   import now_as_str
from glass.smp import proprndcells_to_rst
from glass.wenv.grs import run_grass

In [ ]:
# Create GRASS GIS Session

ws, loc = os.path.dirname(out_rst), f"loc_{now_as_str()}"

grsb = run_grass(ws, location=loc, srs=ref_raster)
    
import grass.script.setup as gsetup
    
gsetup.init(grsb, ws, loc, 'PERMANENT')

In [ ]:
from glass.it.shp import shp_to_grs
from glass.it.rst   import rst_to_grs, grs_to_rst
from glass.dtt.torst import grsshp_to_grsrst
from glass.rst.alg  import grsrstcalc

In [ ]:
# Add Train ESRI Shapefile to GRASS GIS
train_shp = shp_to_grs(train_ref, fprop(train_ref, 'fn'))

# Train to Raster
train_rst = grsshp_to_grsrst(train_shp, class_col, f"rst_{train_shp}")

In [ ]:
# Remove clouds if necessary

if clouds:
    _clouds = rst_to_grs(clouds, fprop(clouds, 'fn'))
    
    train_rst = grsrstcalc(
        f"{train_rst} + {_clouds}", f"f_{train_rst}",
        api='grass'
    )

In [ ]:
# Export result
tres = grs_to_rst(train_rst, os.path.join(
    ws, loc,
    f"tmp_{fprop(out_rst, 'fn')}.tif"
), is_int=True)

# Select random sample
proprndcells_to_rst(
    tres, class_proportion, out_rst,
    sample_dim, cls_sample_min=cls_sample_min
)